# $C$ Class Sentiment Analysis 
In this file I will evalute model when we have mutliple class instead of 0 and 1 if we have $C$ class output. so if we have more than two  examples in output require a $C$ dimensional vector and where $C$ is the number of classes.

In [11]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random

In [12]:
Seed = 1234
torch.manual_seed(Seed)
torch.backends.cudnn.deterministic = True

Text = data.Field(tokenize= 'spacy', tokenizer_language='en_core_web_sm',)
Label = data.LabelField()

train_data, test_data = datasets.TREC.splits(Text, Label, fine_grained = False) # instead of IMDB dataset i use TREC
train_data, val_data = train_data.split(random_state = random.seed(Seed))

In [13]:
# Looking training example
vars(train_data[-1])

{'label': 'DESC', 'text': ['What', 'is', 'a', 'Cartesian', 'Diver', '?']}

In [14]:
Max_Vocab_Size = 25_000

Text.build_vocab(train_data, max_size = Max_Vocab_Size,
                 vectors = "glove.6B.100d",
                 unk_init = torch.Tensor.normal_)

Label.build_vocab(train_data)

In [15]:
#Dataset consist of 6 labels which correspond to 6 types of questions
print(Label.vocab.stoi)

# HUM about humans realated questions
# ENTY about entities realated questions
# DESC for a description questions
# NUM the answer is numerical for questions  
# LOC for questions where the answer is a location
# ABBR for questions asking about abbreviations


defaultdict(None, {'HUM': 0, 'ENTY': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [16]:
Batch_Size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size = Batch_Size,
    device = device
)

In [30]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,  dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        text = text.permute(1, 0)
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
                
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))

        return self.fc(cat)

In [31]:
INPUT_dim = len(Text.vocab)
EMBEDDING_dim = 100
N_filters = 100
FILTER_size = [2,3,4]
OUTPUT_dim = len(Label.vocab)
DROPOUT = 0.5
PAD_idx = Text.vocab.stoi[Text.pad_token]

model = CNN(INPUT_dim, EMBEDDING_dim, N_filters, FILTER_size, OUTPUT_dim, DROPOUT, PAD_idx)

In [33]:
def count_parameters(model):

  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The Model has {count_parameters(model):,} trainable parameters')

The Model has 842,406 trainable parameters


In [34]:
#loading pre trained mode

pretranined_model = Text.vocab.vectors

model.embedding.weight.data.copy_(pretranined_model)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.3110, -0.3398,  1.0308,  ...,  0.5317,  0.2836, -0.0640],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631],
        [ 0.4306,  1.2011,  0.0873,  ...,  0.8817,  0.3722,  0.3458]])

In [35]:
Unk_idx = Text.vocab.stoi[Text.unk_token]

model.embedding.weight.data[Unk_idx] = torch.zeros(EMBEDDING_dim)
model.embedding.weight.data[PAD_idx] = torch.zeros(EMBEDDING_dim)

In [44]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [39]:
def categorical_accuracy(preds, y):

    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    
    return acc

In [45]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [47]:
N_epochs = 5

best_val_loss = float('inf')

for epoch in range(N_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.292 | Train Acc: 49.67%
	 Val. Loss: 0.918 |  Val. Acc: 69.37%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.850 | Train Acc: 69.39%
	 Val. Loss: 0.738 |  Val. Acc: 73.99%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.658 | Train Acc: 76.82%
	 Val. Loss: 0.628 |  Val. Acc: 77.26%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.495 | Train Acc: 83.15%
	 Val. Loss: 0.549 |  Val. Acc: 80.62%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.372 | Train Acc: 88.42%
	 Val. Loss: 0.493 |  Val. Acc: 82.74%


In [48]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.410 | Test Acc: 86.72%


In [50]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [Text.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [55]:
pred_class = predict_class(model, "Who is Alex?")
print(f'Predicted class is: {pred_class} = {Label.vocab.itos[pred_class]}')

Predicted class is: 0 = HUM


In [58]:
pred_class = predict_class(model, "How many hours are in seven hundred days?")
print(f'Predicted class is: {pred_class} = {Label.vocab.itos[pred_class]}')

Predicted class is: 3 = NUM


In [60]:
pred_class = predict_class(model, "What continent is italy in?")
print(f'Predicted class is: {pred_class} = {Label.vocab.itos[pred_class]}')

Predicted class is: 4 = LOC


In [61]:
pred_class = predict_class(model, "What does BTW stand for?")
print(f'Predicted class is: {pred_class} = {Label.vocab.itos[pred_class]}')

Predicted class is: 5 = ABBR
